In [1]:
# Importing Libaries
import re
import unicodedata
import tweepy
import pandas as pd

In [2]:
# Set up your Twitter API credentials
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

In [3]:
# Authenticate with Twitter API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Create an API object
api = tweepy.API(auth, wait_on_rate_limit=True)

# No of tweets to scrape
num_tweets = 1000

In [4]:
# Function to scrape user tweets
def scrape_user_tweets(usernames, num_tweets):
    """Scrapes user tweets from twitter based on the provided entities
    """
    
    tweets = []
    for username in usernames:
        user_tweets = tweepy.Cursor(api.user_timeline, screen_name=username, tweet_mode="extended").items(num_tweets)
        for tweet in user_tweets:
            # Fetching reply count
            status = api.get_status(tweet.id)
            reply_count = status.reply_count if hasattr(status, "reply_count") else 0
            
            data = {
                'Tweet': tweet.full_text if hasattr(tweet, "full_text") else tweet.text,
                'Date': tweet.created_at.date(),
                'Time': tweet.created_at.time(),
                'Twitter Handle': tweet.user.screen_name
            }
            tweets.append(data)
    return tweets

# Function to scrape hashtag tweets
def scrape_hashtag_tweets(hashtags, num_tweets):
    """Scrapes based on the hastag tweets 
    """
    
    tweets = []
    for hashtag in hashtags:
        hashtag_tweets = tweepy.Cursor(api.search_tweets, q=hashtag, lang="en", tweet_mode="extended").items(num_tweets)
        for tweet in hashtag_tweets:
            # Fetching reply count
            status = api.get_status(tweet.id)
            reply_count = status.reply_count if hasattr(status, "reply_count") else 0
            
            data = {
                'Tweet': tweet.full_text if hasattr(tweet, "full_text") else tweet.text,
                'Date': tweet.created_at.date(),
                'Time': tweet.created_at.time(),
                'User Name': tweet.user.name,
                'Twitter Handle': tweet.user.screen_name,
                'Location': tweet.user.location,
                'Followers Count': tweet.user.followers_count,
                'Following Count': tweet.user.friends_count,
                'Profile URL': tweet.user.url,
                'Profile Image URL': tweet.user.profile_image_url,
                'Profile Banner URL': tweet.user.profile_banner_url if 'profile_banner_url' in dir(tweet.user) else None,
                'Account Creation Date': tweet.user.created_at.date(),
                'Retweet Count': tweet.retweet_count,
                'Favorite Count': tweet.favorite_count,
                'Reply Count': reply_count,
                'Hashtags': [hashtag['text'] for hashtag in tweet.entities['hashtags']],
                'User Mentions': [mention['screen_name'] for mention in tweet.entities['user_mentions']],
                'Media URLs': [media['media_url'] for media in tweet.entities.get('media', [])],
                'Tweet URL': f"https://twitter.com/{tweet.user.screen_name}/status/{tweet.id}",
                'Coordinates': tweet.coordinates,
                'Language': tweet.lang,
                'Source': tweet.source,
                'Is Retweet': hasattr(tweet, 'retweeted_status'),
                'Is Quote Status': hasattr(tweet, 'quoted_status'),
                'Quoted Status': tweet.quoted_status.full_text if hasattr(tweet, 'quoted_status') else None,
                'In Reply To User': tweet.in_reply_to_screen_name,
                'Place': tweet.place.full_name if tweet.place else None,
                'Entities': tweet.entities,
                'Is Verified User': tweet.user.verified,
                'Is Retweeted': tweet.retweeted,
                'Is Favorited': tweet.favorited
            }
            tweets.append(data)
            
    return tweets

In [5]:
# Define the target users and hashtags to scrape
target_users = ["LloydsBank", "MyMaybank", "ZenithBank", "NatWest_Help", "Nedbank", "talktoBOI"]
hashtags = ["LloydsBank", "Maybank", "ZenithBank", "Natwest", "Nedbank", "BankOfIreland"]

# Scrape tweets from target users
user_tweets = scrape_user_tweets(target_users, num_tweets)

# Convert user tweets to a dataframe
user_tweets_df = pd.DataFrame(user_tweets)

# Scrape tweets with hashtags
hashtag_tweets = scrape_hashtag_tweets(hashtags, num_tweets)

# Convert hashtag tweets to a dataframe
hashtag_tweets_df = pd.DataFrame(hashtag_tweets)

Rate limit reached. Sleeping for: 491
Rate limit reached. Sleeping for: 587
Rate limit reached. Sleeping for: 596
Rate limit reached. Sleeping for: 595
Rate limit reached. Sleeping for: 600
Rate limit reached. Sleeping for: 603
Rate limit reached. Sleeping for: 600
Rate limit reached. Sleeping for: 585
Rate limit reached. Sleeping for: 579
Rate limit reached. Sleeping for: 585
Rate limit reached. Sleeping for: 587


In [8]:
user_tweets_df.to_csv('user_tweets.csv', index=False)

In [9]:
hashtag_tweets_df.to_csv('user_hashtag.csv', index=False)